In [10]:
import torch
from dataloader import EmbeddingsDataloader
from torch.utils.data import DataLoader
models = torch.load('saved_models_singleperceptron_best_performance.save')

In [8]:
model = models[-1][2]
frames_per_datapoint = int(models[-1][1].split(' ')[2].split(')')[0])
print(frames_per_datapoint)

64


In [12]:
device = 'mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu'

test_dataset = EmbeddingsDataloader(width=frames_per_datapoint, mode='test', overlap=False)
test_dataloader = DataLoader(test_dataset, batch_size=512)
def test():
    with torch.no_grad():
        total_attempts = 0
        correct = 0
        classified = []
        ground_truth = []
        for x, y in test_dataloader:
            x, y = x.to(device), y.to(device)
            res = model(x)
            # label = one_hot(y[:,-1].type(torch.int64), num_classes=9).float()
            total_attempts += x.shape[0]
            correct += float((y[:,-1] == torch.argmax(res, dim=-1)).sum())

            ground_truth.append(torch.one_hot(y[:,-1].type(torch.int64), num_classes=9).float().cpu().numpy())
            classified.append(res.cpu().numpy())
        return round(correct / total_attempts, 3), classified, ground_truth

testacc, y_hat, y = test()

TypeError: empty(): argument 'size' must be tuple of ints, but found element of type Tensor at pos 2